### Setup

In [1]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.linear_model import ElasticNetCV, LinearRegression, Lars, ElasticNet
from sklearn.svm import LinearSVR, SVR
from sklearn.preprocessing import StandardScaler, FunctionTransformer, PolynomialFeatures
from sklearn.model_selection import cross_validate, train_test_split, RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from vb_helper import VBHelper,shrinkBigKTransformer,logminus_T,exp_T,logminplus1_T,none_T, logp1_T

In [2]:
import daal4py.sklearn
daal4py.sklearn.patch_sklearn()

Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


In [3]:
test_share=0.2 
cv_folds=10
cv_reps=10
cv_count=cv_folds*cv_reps
rs=1 # random_state for reproducibility
vbhelper=VBHelper(test_share,cv_folds,cv_reps,cv_count,rs)

##### Example Dataset

In [4]:
X, y, w = make_regression(n_samples=300,
                          n_features=8, # x variables generated and returned 
                          n_informative=2, # x variables included in the actual model of y
                          effective_rank=2, # make less than n_informative for multicollinearity
                          coef=True,
                          noise=3,
                          random_state=rs,
                          bias=1)

#xt=np.product(X[:,0:2],axis=-1)
for i in range(1,5,5):
    sgn_mult=3**i*(-1)**i
    y+=sgn_mult*np.product(10*X[:,i:i+2],axis=-1)
print(y.shape)
#xtnorm=xt/np.sum(xt)
#print(xtnorm.shape)
y=np.exp((y-np.min(y))/10+1)
print(y)


(300,)
[ 5.92204312  8.50829925  9.29127265 11.68951601  9.12258936  5.49719264
 14.3562217   9.24413212  5.49112774 18.61125476  7.56701548 13.5675252
  7.28330449  6.11116954 10.19300877 12.47879443 17.83837868  8.24769078
 10.86826015  8.832442   15.02564369  7.59370547  9.01252021  7.39590944
 12.61375     5.38415542  5.75039776  9.25207432 10.13815771  7.50420653
 14.46515236  7.40392825  8.35857582  4.8762096  11.81714571  9.62465031
 16.72547231 13.34529152  9.21522548 11.29091934 11.14484074  8.40199541
  6.41762653 10.69788092  4.96805857  9.58393763  6.38961059 14.36975938
  6.84925263  7.47084725  9.88212888 12.09678597  7.79288779 14.75374146
  5.81990046 14.76559453  7.74378062  5.15092576  4.49746358  8.90075735
  7.96764892  7.52072818  7.13820281 10.54947602 21.81560383  3.66825392
  9.08647989 11.77309092  5.55925765  6.18225542  8.48883634  9.71042079
 13.06677646  8.63463203 15.6807087   9.69461892 13.04265622  5.49081403
  6.96509038  9.90620466  9.76921876  8.23196

In [5]:
# add interaction terms

In [6]:
if test_share:
     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_share, random_state=rs)
else:
    X_train, X_test, y_train, y_test = (X, None, y, None)

In [7]:
n,k=X_train.shape

max_k=n//2

vbhelper.max_k=max_k

In [8]:
# use lambda to make a callable object for creating new models, but with args set already
# may be unnecessary due to sklearn cloning


linear_regression=lambda: make_pipeline(StandardScaler(),LinearRegression(fit_intercept=1)) 
linear_regression_lars=lambda: make_pipeline(StandardScaler(),shrinkBigKTransformer(max_k=max_k),LinearRegression()) #https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lars.html
elastic_net =lambda: make_pipeline(StandardScaler(), ElasticNetCV())
linear_svr =lambda: make_pipeline(StandardScaler(),LinearSVR(random_state=rs,tol=1e-4,max_iter=5000,C=1))
rbf_svr=lambda: make_pipeline(StandardScaler(),SVR(kernel='rbf',tol=1e-4,max_iter=5000, C=1))
gradient_boosting_reg=lambda: make_pipeline(GradientBoostingRegressor())

g_pts=3 # grid points for gridsearchcv param_grid 
linear_svr = Pipeline(steps=[('scaler',StandardScaler()),('lin_svr',LinearSVR(random_state=0,tol=1e-4,max_iter=10000))])
lin_svr_param_grid={'lin_svr__C':np.logspace(-2,2,g_pts)}
linear_svr_cv=lambda: GridSearchCV(linear_svr,param_grid=lin_svr_param_grid)

rbf_svr=Pipeline(steps=[('scaler',StandardScaler()),('rbf_svr',SVR(kernel='rbf',tol=1e-4,max_iter=10000, cache_size=2*10**3))])
rbf_svr_param_grid={'rbf_svr__C':np.logspace(-2,2,g_pts),'rbf_svr__gamma':np.logspace(-1,0.5,g_pts)} 
rbf_svr_cv=lambda: GridSearchCV(rbf_svr,param_grid=rbf_svr_param_grid)


In [9]:
transformer_list=[none_T(),logp1_T ()]#[logminplus1_T(),none_T(),logminus_T()]#exp_T()] # imported...
lin_reg_y_t_pipe=Pipeline(steps=[('ttr',TransformedTargetRegressor(regressor=linear_regression_lars()))])
lin_reg_y_t_param_grid={'ttr__transformer':transformer_list}
lin_reg_y_transform=lambda: GridSearchCV(lin_reg_y_t_pipe,param_grid=lin_reg_y_t_param_grid)

#### add PolynomialFeatures() to gridsearch
#### and try shrinking the number of parameters

In [10]:
steps=[
    ('scaler',StandardScaler()),
    #('shrink_k1',shrinkBigKTransformer(selector='elastic-net')), # retain a subset of the best original variables
    ('polyfeat',PolynomialFeatures(interaction_only=1)), # create interactions among them
    ('shrink_k2',shrinkBigKTransformer(selector='elastic-net')), # pick from all of those options
    ('pca',PCA()),
    ('reg',linear_regression())]

inner_params={'polyfeat__degree':[2],
                        'pca__n_components':np.logspace(np.log10(0.5),0,5,endpoint=False),
                        }

inner_cv=RepeatedKFold(n_splits=5, n_repeats=1, random_state=rs)
X_T_pipe=GridSearchCV(Pipeline(steps=steps),param_grid=inner_params,cv=inner_cv)

 

Y_T_X_T_pipe=Pipeline(steps=[('ttr',TransformedTargetRegressor(regressor=X_T_pipe))])
Y_T__param_grid={'ttr__transformer':transformer_list}
lin_reg_Xy_PCA_transform=lambda: GridSearchCV(Y_T_X_T_pipe,param_grid=Y_T__param_grid,cv=inner_cv)



In [11]:
estimator_dict={'linear-regression':linear_regression,
                #'linear-regression-lars':linear_regression_lars,
                #'lin_reg_y_transform':lin_reg_y_transform,
                'lin_reg_Xy_PCA_transform':lin_reg_Xy_PCA_transform,
                'elastic-net':elastic_net, }
                #'linear-svr-cv':linear_svr_cv, }
                #'rbf-svr-cv':rbf_svr_cv, 
                #'gradient-boosting-reg':gradient_boosting_reg}
vbhelper.estimator_dict=estimator_dict
model_dict={key:val() for key,val in estimator_dict.items()} # they will be models once .fit is called

In [12]:
scorer_list=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'] #cross_validate wants strings
cv=RepeatedKFold(n_splits=cv_folds, n_repeats=cv_reps, random_state=rs) # define separately to ensure same cv data used for each model
vbhelper.scorer_list=scorer_list
# allow/generate water quality thresholds for stratified kfold sub-sampling to ensure cross-validation folds have full range of water quality

In [13]:
#simple=[est.fit(X_train,y_train) for name,est in model_dict.items()]


In [ ]:
cv_results={estimator_name:cross_validate(model, X_train, y_train, return_estimator=True, scoring=scorer_list, cv=cv)
            for estimator_name,model in model_dict.items()}
# replace with a loop in order to save the residuals for a graph?

### graphs and table to summarize results

In [ ]:
cv_score_dict={}
cv_score_dict_means={}
for idx,(estimator_name,result) in enumerate(cv_results.items()):
    #cv_estimators=result['estimator']
    model_idx_scoredict={scorer:result[f'test_{scorer}'] for scorer in scorer_list}# fstring bc how cross_validate stores list of metrics
    cv_score_dict[estimator_name]=model_idx_scoredict 
    model_idx_mean_scores={scorer:np.mean(scores) for scorer,scores in model_idx_scoredict.items()}
    cv_score_dict_means[estimator_name]=model_idx_mean_scores

In [ ]:
for scorer in scorer_list:
    print(f'scores for scorer: {scorer}:')
    for estimator_name in model_dict:
        print(f'    {estimator_name}:{cv_score_dict_means[estimator_name][scorer]}')

In [ ]:
vbhelper.plotCVScores(cv_score_dict,sort=1)

In [ ]:
# create a similar plot showing residuals from the cv models for each value of y. 
# needs to be scatterplot or histogram since there will be (folds-1)*repeats predictions of each value of y.

-----------------------------
### User chooses Linear Regression with LARS variable selection!


In [ ]:
final_estimator_name='linear-regression-lars'

In [ ]:
def printTestandCVScores(estimator_name,cv_score_dict_means):
    model=estimator_dict[estimator_name]()
    model.fit(X_train,y_train)
    if test_share:
        y_test_hat=model.predict(X_test)
        print(f'test set: negative-mse={-mean_squared_error(y_test,y_test_hat)}')
    for scorer in scorer_list:
        print(f'cv avg: {scorer}= {cv_score_dict_means[estimator_name][scorer]}')
    try:
        print('coefficients: ',model[-1].coef_)
        print('intercept: ',model[-1].intercept_)
        #print('\n','original positions: ',model[-2].col_select)
    except:
        pass

In [ ]:
for name in estimator_dict.keys():
    print(name)
    printTestandCVScores(final_estimator_name,cv_score_dict_means)

In [ ]:
printTestandCVScores('elastic-net',cv_score_dict_means)
# fits better but soooo many coefficients

In [ ]:
printTestandCVScores('lin_reg_Xy_transform',cv_score_dict_means)